# NN with OneHotEncode Target Values
Four Binary outputs

Performs better than NN with LabelEncode Target Values

In [1]:
#Import Dependencies

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
from sqlalchemy import create_engine, func
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [2]:
#SOURCE: https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm
import psycopg2

#establishing the connection
conn = psycopg2.connect(
   database="Body_Performance", user='postgres', password='Cheese6132', host='localhost', port= '5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Executing an MYSQL function using the execute() method
cursor.execute("select version()")

# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)



Connection established to:  ('PostgreSQL 12.13, compiled by Visual C++ build 1914, 64-bit',)


In [3]:
body_df = pd.read_sql('select * from body_performance', con=conn)
body_df.head()

,age,gender,height_cm,weight_kg,body_fat_pct,diastolic,systolic,gripforce,sit_and_bend_forward_cm,sit_ups_counts,broad_jump_cm,ranking
0,27,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B


In [4]:
#Closing the connection
conn.close()
print("Closed connection established to: ",data)


Closed connection established to:  ('PostgreSQL 12.13, compiled by Visual C++ build 1914, 64-bit',)


In [5]:
# OneHotEncoder or Get_dummies on Gender
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

encode1 = pd.DataFrame(enc.fit_transform(body_df.gender.values.reshape(-1,1)))

encode1.columns = enc.get_feature_names_out(['gender'])
encode1.head()


,gender_F,gender_M
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [6]:
#If keeping class/level in one column as Y, Encode with dictionary values
#ranking_num = {"A":0, "B":1, "C":2, "D":3}
#body_df["ranking_num"] = body_df["ranking"].apply(lambda x: ranking_num[x])
#body_df.head()


In [6]:
#Using oneHotEncoder for class into 4 separate

encode2 = pd.DataFrame(enc.fit_transform(body_df.ranking.values.reshape(-1,1)))

encode2.columns = enc.get_feature_names_out(["ranking"])
encode2.head()

,ranking_A,ranking_B,ranking_C,ranking_D
0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0


In [7]:
#Drop columns and merge encoded df
#Ignoring encode2 for now
body_df = body_df.drop(["gender", "ranking"], axis=1)

body_df2 = body_df.merge(encode1, left_index=True, right_index=True)
body_df2.head()


,age,height_cm,weight_kg,body_fat_pct,diastolic,systolic,gripforce,sit_and_bend_forward_cm,sit_ups_counts,broad_jump_cm,gender_F,gender_M
0,27,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,0.0,1.0
1,25,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,0.0,1.0
2,31,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,0.0,1.0
3,32,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,0.0,1.0
4,28,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,0.0,1.0


In [8]:
#Set X, y 
X = body_df2.values
    #X = X.drop("level_num", axis=1)
    #y = body_df2["level_num"].ravel()
y = encode2.values
print(X.shape, y.shape)


(13393, 12) (13393, 4)


In [9]:
y[:10]

array([[0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]])

In [39]:
#Split train/test data and scale using Standard Scaler at this time (mean of 0).  #Should we use MinMax(0-1)
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

scaler = StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_train_scaled.shape, X_test_scaled.shape)

(10044, 12) (3349, 12)


In [40]:
# Define the model - deep neural net...length of outputs/inputs
num_input_features = len(X_train_scaled[0])
num_output_features = len(y_train[0])
print(num_input_features)
print(num_output_features)

12
4


In [41]:
# Define the model - deep neural net
# Try with 2 hidden layers first but will try reduced. Look at different activation functions?
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 18
hidden_nodes_layer3 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer1, input_dim=num_input_features,
    activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer  #Will need to round or should this be sigmoid?
nn.add(tf.keras.layers.Dense(units=num_output_features, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 24)                312       
                                                                 
 dense_26 (Dense)            (None, 18)                450       
                                                                 
 dense_27 (Dense)            (None, 12)                228       
                                                                 
 dense_28 (Dense)            (None, 4)                 52        
                                                                 
Total params: 1,042
Trainable params: 1,042
Non-trainable params: 0
_________________________________________________________________


In [43]:
# Compile the model binary vs categorical. Categorical for multi-output  --https://machinelearningmastery.com/deep-learning-models-for-multi-output-regression/
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
#nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [46]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=400)

Epoch 1/400
314/314 [==============================] - 0s 919us/step - loss: 0.2526 - accuracy: 0.7711
Epoch 2/400
314/314 [==============================] - 0s 952us/step - loss: 0.2523 - accuracy: 0.7689
Epoch 3/400
314/314 [==============================] - 0s 962us/step - loss: 0.2522 - accuracy: 0.7679
Epoch 4/400
314/314 [==============================] - 0s 1ms/step - loss: 0.2523 - accuracy: 0.7692
Epoch 5/400
314/314 [==============================] - 0s 950us/step - loss: 0.2520 - accuracy: 0.7708
Epoch 6/400
314/314 [==============================] - 0s 886us/step - loss: 0.2519 - accuracy: 0.7675
Epoch 7/400
314/314 [==============================] - 0s 792us/step - loss: 0.2524 - accuracy: 0.7667
Epoch 8/400
314/314 [==============================] - 0s 786us/step - loss: 0.2517 - accuracy: 0.7696
Epoch 9/400
314/314 [==============================] - 0s 773us/step - loss: 0.2520 - accuracy: 0.7686
Epoch 10/400
314/314 [==============================] - 0s 781us/step - los

314/314 [==============================] - 0s 785us/step - loss: 0.2468 - accuracy: 0.7735
Epoch 80/400
314/314 [==============================] - 0s 772us/step - loss: 0.2464 - accuracy: 0.7729
Epoch 81/400
314/314 [==============================] - 0s 843us/step - loss: 0.2469 - accuracy: 0.7686
Epoch 82/400
314/314 [==============================] - 0s 830us/step - loss: 0.2464 - accuracy: 0.7725
Epoch 83/400
314/314 [==============================] - 0s 810us/step - loss: 0.2463 - accuracy: 0.7711
Epoch 84/400
314/314 [==============================] - 0s 791us/step - loss: 0.2462 - accuracy: 0.7723
Epoch 85/400
314/314 [==============================] - 0s 777us/step - loss: 0.2462 - accuracy: 0.7730
Epoch 86/400
314/314 [==============================] - 0s 765us/step - loss: 0.2460 - accuracy: 0.7724
Epoch 87/400
314/314 [==============================] - 0s 789us/step - loss: 0.2465 - accuracy: 0.7704
Epoch 88/400
314/314 [==============================] - 0s 777us/step - loss:

314/314 [==============================] - 0s 833us/step - loss: 0.2423 - accuracy: 0.7793
Epoch 158/400
314/314 [==============================] - 0s 776us/step - loss: 0.2418 - accuracy: 0.7781
Epoch 159/400
314/314 [==============================] - 0s 913us/step - loss: 0.2420 - accuracy: 0.7802
Epoch 160/400
314/314 [==============================] - 0s 896us/step - loss: 0.2421 - accuracy: 0.7790
Epoch 161/400
314/314 [==============================] - 0s 877us/step - loss: 0.2419 - accuracy: 0.7815
Epoch 162/400
314/314 [==============================] - 0s 804us/step - loss: 0.2422 - accuracy: 0.7775
Epoch 163/400
314/314 [==============================] - 0s 802us/step - loss: 0.2423 - accuracy: 0.7786
Epoch 164/400
314/314 [==============================] - 0s 791us/step - loss: 0.2422 - accuracy: 0.7752
Epoch 165/400
314/314 [==============================] - 0s 802us/step - loss: 0.2418 - accuracy: 0.7817
Epoch 166/400
314/314 [==============================] - 0s 935us/ste

314/314 [==============================] - 0s 863us/step - loss: 0.2389 - accuracy: 0.7847
Epoch 236/400
314/314 [==============================] - 0s 850us/step - loss: 0.2391 - accuracy: 0.7814
Epoch 237/400
314/314 [==============================] - 0s 772us/step - loss: 0.2386 - accuracy: 0.7809
Epoch 238/400
314/314 [==============================] - 0s 853us/step - loss: 0.2390 - accuracy: 0.7825
Epoch 239/400
314/314 [==============================] - 0s 845us/step - loss: 0.2386 - accuracy: 0.7832
Epoch 240/400
314/314 [==============================] - 0s 784us/step - loss: 0.2390 - accuracy: 0.7819
Epoch 241/400
314/314 [==============================] - 0s 875us/step - loss: 0.2388 - accuracy: 0.7845
Epoch 242/400
314/314 [==============================] - 0s 831us/step - loss: 0.2386 - accuracy: 0.7845
Epoch 243/400
314/314 [==============================] - 0s 798us/step - loss: 0.2387 - accuracy: 0.7816
Epoch 244/400
314/314 [==============================] - 0s 841us/ste

314/314 [==============================] - 0s 793us/step - loss: 0.2367 - accuracy: 0.7849
Epoch 314/400
314/314 [==============================] - 0s 770us/step - loss: 0.2364 - accuracy: 0.7855
Epoch 315/400
314/314 [==============================] - 0s 862us/step - loss: 0.2364 - accuracy: 0.7881
Epoch 316/400
314/314 [==============================] - 0s 822us/step - loss: 0.2357 - accuracy: 0.7879
Epoch 317/400
314/314 [==============================] - 0s 798us/step - loss: 0.2368 - accuracy: 0.7870
Epoch 318/400
314/314 [==============================] - 0s 863us/step - loss: 0.2362 - accuracy: 0.7861
Epoch 319/400
314/314 [==============================] - 0s 833us/step - loss: 0.2360 - accuracy: 0.7857
Epoch 320/400
314/314 [==============================] - 0s 852us/step - loss: 0.2358 - accuracy: 0.7867
Epoch 321/400
314/314 [==============================] - 0s 801us/step - loss: 0.2361 - accuracy: 0.7858
Epoch 322/400
314/314 [==============================] - 0s 816us/ste

314/314 [==============================] - 0s 785us/step - loss: 0.2338 - accuracy: 0.7895
Epoch 392/400
314/314 [==============================] - 0s 776us/step - loss: 0.2337 - accuracy: 0.7874
Epoch 393/400
314/314 [==============================] - 0s 780us/step - loss: 0.2336 - accuracy: 0.7897
Epoch 394/400
314/314 [==============================] - 0s 782us/step - loss: 0.2339 - accuracy: 0.7872
Epoch 395/400
314/314 [==============================] - 0s 773us/step - loss: 0.2335 - accuracy: 0.7901
Epoch 396/400
314/314 [==============================] - 0s 789us/step - loss: 0.2335 - accuracy: 0.7903
Epoch 397/400
314/314 [==============================] - 0s 783us/step - loss: 0.2335 - accuracy: 0.7890
Epoch 398/400
314/314 [==============================] - 0s 851us/step - loss: 0.2337 - accuracy: 0.7893
Epoch 399/400
314/314 [==============================] - 0s 806us/step - loss: 0.2342 - accuracy: 0.7885
Epoch 400/400
314/314 [==============================] - 0s 788us/ste

In [50]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

105/105 - 0s - loss: 0.3088 - accuracy: 0.7328 - 69ms/epoch - 656us/step
Loss: 0.30884209275245667, Accuracy: 0.7327560186386108


# second model test

In [33]:
# New Layer node count

nhidden_nodes_layer1 =  18
nhidden_nodes_layer2 = 10
nhidden_nodes_layer3 = 10
nhidden_nodes_layer4 = 6

In [34]:


nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(
    units=nhidden_nodes_layer1, input_dim=num_input_features,
    activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=nhidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=nhidden_nodes_layer3, activation="sigmoid"))

# Fourth hidden layer
nn2.add(tf.keras.layers.Dense(units=nhidden_nodes_layer4, activation="sigmoid"))

# Output layer  #Will need to round or should this be sigmoid?
nn2.add(tf.keras.layers.Dense(units=num_output_features, activation="softmax"))

# Check the structure of the model
nn2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 18)                234       
                                                                 
 dense_17 (Dense)            (None, 10)                190       
                                                                 
 dense_18 (Dense)            (None, 10)                110       
                                                                 
 dense_19 (Dense)            (None, 6)                 66        
                                                                 
 dense_20 (Dense)            (None, 4)                 28        
                                                                 
Total params: 628
Trainable params: 628
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [36]:
# Train the model
fit_model = nn2.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
314/314 [==============================] - 1s 785us/step - loss: 0.5816 - accuracy: 0.2487
Epoch 2/200
314/314 [==============================] - 0s 769us/step - loss: 0.5624 - accuracy: 0.2513
Epoch 3/200
314/314 [==============================] - 0s 773us/step - loss: 0.5624 - accuracy: 0.2535
Epoch 4/200
314/314 [==============================] - 0s 776us/step - loss: 0.5623 - accuracy: 0.2588
Epoch 5/200
314/314 [==============================] - 0s 777us/step - loss: 0.5617 - accuracy: 0.2974
Epoch 6/200
314/314 [==============================] - 0s 770us/step - loss: 0.5559 - accuracy: 0.4018
Epoch 7/200
314/314 [==============================] - 0s 765us/step - loss: 0.5170 - accuracy: 0.4890
Epoch 8/200
314/314 [==============================] - 0s 776us/step - loss: 0.4752 - accuracy: 0.4880
Epoch 9/200
314/314 [==============================] - 0s 768us/step - loss: 0.4591 - accuracy: 0.5142
Epoch 10/200
314/314 [==============================] - 0s 776us/step - l

314/314 [==============================] - 0s 845us/step - loss: 0.3642 - accuracy: 0.6291
Epoch 80/200
314/314 [==============================] - 0s 797us/step - loss: 0.3643 - accuracy: 0.6268
Epoch 81/200
314/314 [==============================] - 0s 790us/step - loss: 0.3643 - accuracy: 0.6303
Epoch 82/200
314/314 [==============================] - 0s 798us/step - loss: 0.3637 - accuracy: 0.6269
Epoch 83/200
314/314 [==============================] - 0s 781us/step - loss: 0.3648 - accuracy: 0.6292
Epoch 84/200
314/314 [==============================] - 0s 792us/step - loss: 0.3636 - accuracy: 0.6302
Epoch 85/200
314/314 [==============================] - 0s 780us/step - loss: 0.3636 - accuracy: 0.6278
Epoch 86/200
314/314 [==============================] - 0s 791us/step - loss: 0.3636 - accuracy: 0.6309
Epoch 87/200
314/314 [==============================] - 0s 771us/step - loss: 0.3636 - accuracy: 0.6280
Epoch 88/200
314/314 [==============================] - 0s 776us/step - loss:

314/314 [==============================] - 0s 910us/step - loss: 0.3323 - accuracy: 0.6857
Epoch 158/200
314/314 [==============================] - 0s 937us/step - loss: 0.3320 - accuracy: 0.6854
Epoch 159/200
314/314 [==============================] - 0s 830us/step - loss: 0.3314 - accuracy: 0.6853
Epoch 160/200
314/314 [==============================] - 0s 825us/step - loss: 0.3313 - accuracy: 0.6830
Epoch 161/200
314/314 [==============================] - 0s 838us/step - loss: 0.3312 - accuracy: 0.6869
Epoch 162/200
314/314 [==============================] - 0s 829us/step - loss: 0.3298 - accuracy: 0.6898
Epoch 163/200
314/314 [==============================] - 0s 832us/step - loss: 0.3303 - accuracy: 0.6888
Epoch 164/200
314/314 [==============================] - 0s 931us/step - loss: 0.3303 - accuracy: 0.6857
Epoch 165/200
314/314 [==============================] - 0s 916us/step - loss: 0.3287 - accuracy: 0.6888
Epoch 166/200
314/314 [==============================] - 0s 808us/ste

In [37]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#One Hidden Layer lowered accuracy and inreased loss. 

105/105 - 0s - loss: 0.3231 - accuracy: 0.6999 - 139ms/epoch - 1ms/step
Loss: 0.3230709135532379, Accuracy: 0.6999104022979736


In [ ]:
feature_importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

In [ ]:
#Accuracy score and confusion Matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

predictions = rf_model.predict(X_test_scaled)
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

In [ ]:
# printing confusion matrix but error:      Shape of passed values is (4, 4), indices imply (2, 2) Needs to be resolved
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"])
display(cm_df)

In [ ]:
#Classification report for random forest

print("Classification Report")
print(classification_report(y_test, predictions))